In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import csv

def summarize_columns(df):
    print(pd.DataFrame([
        (
            c,
            df[c].dtype,
            len(df[c].unique()),
            df[c].memory_usage(deep=True) // (1024**2)
        ) for c in df.columns
    ], columns=['name', 'dtype', 'unique', 'size (MB)']))
    print('Total size:', df.memory_usage(deep=True).sum() / 1024**2, 'MB')

ModuleNotFoundError: No module named 'pyarrow'

### Optimize dataframe ###
To get the optimized dataframe, run the first 3 cells including the import cell. This should write out a new csv file in the data/ folder. Running the optimizer function will take some time (5-10 min) but should only be required once.

To load the new dataframe either use the standard ``pd.read_csv`` or ideally use pyarrow: 

``from pyarrow import csv`` \
``table = csv.read_csv("data/CaseRigshospitalet_optimized.csv")``  \
``df = table.to_pandas()``


In [6]:
table = csv.read_csv("data/CaseRigshospitalet.csv")
df = table.to_pandas()

In [7]:
def optimize_df(df):
    df['Patient ID'] = df['Patient ID'].astype('category')
    df['Patient alder på kontaktstart tidspunkt'] = pd.to_numeric(df['Patient alder på kontaktstart tidspunkt'], errors='coerce').astype('Int16')
    df['Kontakt startdato'] = pd.to_datetime(df['Kontakt startdato'], format='mixed')
    df['Kontakt slutdato'] = pd.to_datetime(df['Kontakt slutdato'], format='mixed')
    df['Kontakttype'] = df['Kontakttype'].astype('category')
    df['Indlæggelsesmåde'] = df['Indlæggelsesmåde'].astype('category')
    df['Patientkontakttype'] = df['Patientkontakttype'].astype('category')
    df['Aktionsdiagnosekode'] = df['Aktionsdiagnosekode'].astype('category')
    df['Bidiagnosekode'] = df['Bidiagnosekode'].astype('category')
    df['Behandlingsansvarlig Afdeling'] = df['Behandlingsansvarlig Afdeling'].astype('category')
    df['Procedure-kode'] = df['Procedure-kode'].astype('category')
    df['Procedure-tillægskoder'] = df['Procedure-tillægskoder'].astype('category')
    df['Procedure udført'] = pd.to_datetime(df['Procedure udført'], format='mixed')
    
    # Ensure conversion runs only if needed
    if df['Kontakt varighed (timer)'].dtype == 'object':  
        df['Kontakt varighed (timer)'] = df['Kontakt varighed (timer)'].str.replace(',', '.').astype('float32')

    df['Besøgstype'] = df['Besøgstype'].astype('category')
    df['Patient køn'] = df['Patient køn'].astype('category')
    df['Patient civilstand'] = df['Patient civilstand'].astype('category')
    df['Patient oprettet på Min SP (J/N)'] = df['Patient oprettet på Min SP (J/N)'].astype('category')
    df['Patient land'] = df['Patient land'].astype('category')
    df['Patient region'] = df['Patient region'].astype('category')
    df['Patient postnummer'] = pd.to_numeric(df['Patient postnummer'], errors='coerce').astype('Int32')
    df['Patient kommune'] = df['Patient kommune'].astype('category')

    return df

# Optimize DataFrame
opt_df = optimize_df(df)

# Convert to PyArrow Table and save as Parquet
table = pa.Table.from_pandas(opt_df)
pq.write_table(table, "data/CaseRigshospitalet_optimized.parquet")

In [8]:
opt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4658066 entries, 0 to 4658065
Data columns (total 24 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   Patient ID                               category      
 1   Patientkontakt ID                        int64         
 2   Patient alder på kontaktstart tidspunkt  Int16         
 3   Kontakt startdato                        datetime64[ns]
 4   Kontakt slutdato                         datetime64[ns]
 5   Kontakttype                              category      
 6   Indlæggelsesmåde                         category      
 7   Patientkontakttype                       category      
 8   Aktionsdiagnosekode                      category      
 9   Bidiagnosekode                           category      
 10  Behandlingsansvarlig Afdeling            category      
 11  Kontakt varighed (timer)                 float32       
 12  Procedure-kode              

In [71]:
summarize_columns(df)

                                       name     dtype   unique  size (MB)
0                                Patient ID  category   331763         46
1                         Patientkontakt ID     int64  1821200         35
2   Patient alder på kontaktstart tidspunkt     Int16      109         13
3                         Kontakt startdato    object   314246        355
4                          Kontakt slutdato    object   283384        355
5                               Kontakttype    object        3        480
6                          Indlæggelsesmåde    object        2        288
7                        Patientkontakttype    object        3        299
8                       Aktionsdiagnosekode    object     8195        276
9                            Bidiagnosekode    object     6416        259
10            Behandlingsansvarlig Afdeling    object       68        596
11                 Kontakt varighed (timer)    object     1462        269
12                           Procedure

In [69]:

df['Patient ID'].unique()[:10]
len(df)

4658066

In [70]:
def optimize_df(df):
    df['Patient ID'] = df['Patient ID'].astype('category')
    df['Patient alder på kontaktstart tidspunkt'] =  pd.to_numeric(df['Patient alder på kontaktstart tidspunkt'], errors='coerce').astype('Int16')
    df['Kontakt startdato'] = pd.to_datetime(df['Kontakt startdato'], format='mixed')
    df['Kontakt slutdato'] = pd.to_datetime(df['Kontakt slutdato'], format='mixed')
    df['Kontakttype'] = df['Kontakttype'].astype('category')
    df['Indlæggelsesmåde'] = df['Indlæggelsesmåde'].astype('category')
    df['Patientkontakttype'] = df['Patientkontakttype'].astype('category')
    df['Aktionsdiagnosekode'] = df['Aktionsdiagnosekode'].astype('category')
    df['Bidiagnosekode'] = df['Bidiagnosekode'].astype('category')
    df['Behandlingsansvarlig Afdeling'] = df['Behandlingsansvarlig Afdeling'].astype('category')
    df['Procedure-kode'] = df['Procedure-kode'].astype('category')
    df['Procedure-tillægskoder'] = df['Procedure-tillægskoder'].astype('category')
    df['Procedure udført'] = pd.to_datetime(df['Procedure udført'], format='mixed')
    
    if df['Kontakt varighed (timer)'].dtype == 'object':  
        df['Kontakt varighed (timer)'] = df['Kontakt varighed (timer)'].str.replace(',', '.').astype('float32')
    df['Besøgstype'] = df['Besøgstype'].astype('category')
    df['Patient køn'] = df['Patient køn'].astype('category')
    df['Patient civilstand'] = df['Patient civilstand'].astype('category')
    df['Patient oprettet på Min SP (J/N)'] = df['Patient oprettet på Min SP (J/N)'].astype('category')
    df['Patient land'] = df['Patient land'].astype('category')
    df['Patient region'] = df['Patient region'].astype('category')
    df['Patient postnummer'] = pd.to_numeric(df['Patient postnummer'], errors='coerce').astype('Int32')
    df['Patient kommune'] = df['Patient kommune'].astype('category')
    return df
    
opt_df = optimize_df(df)
    

KeyboardInterrupt: 

In [ ]:

summarize_columns(opt_df)

                                       name           dtype   unique  \
0                                Patient ID        category   331763   
1                         Patientkontakt ID           int64  1821200   
2   Patient alder på kontaktstart tidspunkt           Int16      109   
3                         Kontakt startdato  datetime64[ns]   314246   
4                          Kontakt slutdato  datetime64[ns]   283384   
5                               Kontakttype        category        3   
6                          Indlæggelsesmåde        category        2   
7                        Patientkontakttype        category        3   
8                       Aktionsdiagnosekode        category     8195   
9                            Bidiagnosekode        category     6416   
10            Behandlingsansvarlig Afdeling        category       68   
11                 Kontakt varighed (timer)         float32     1462   
12                           Procedure-kode        category     

In [ ]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4658066 entries, 0 to 4658065
Data columns (total 24 columns):
 #   Column                                   Dtype         
---  ------                                   -----         
 0   Patient ID                               category      
 1   Patientkontakt ID                        int64         
 2   Patient alder på kontaktstart tidspunkt  Int16         
 3   Kontakt startdato                        datetime64[ns]
 4   Kontakt slutdato                         datetime64[ns]
 5   Kontakttype                              category      
 6   Indlæggelsesmåde                         category      
 7   Patientkontakttype                       category      
 8   Aktionsdiagnosekode                      category      
 9   Bidiagnosekode                           category      
 10  Behandlingsansvarlig Afdeling            category      
 11  Kontakt varighed (timer)                 float32       
 12  Procedure-kode              